##### 1. First we import the libraries we need

In [11]:
import numpy as np
import pandas as pd
import requests
import re

##### 2. Now connect to the API

In [12]:
url = 'https://pokeapi.co/api/v2/pokemon/'

res = requests.get(url)
if res.status_code == 200:
    page = res.json()#We get the json with the data

##### 3. Now let's see the data we collected

In [13]:
page

{'count': 1154,
 'next': 'https://pokeapi.co/api/v2/pokemon/?offset=20&limit=20',
 'previous': None,
 'results': [{'name': 'bulbasaur',
   'url': 'https://pokeapi.co/api/v2/pokemon/1/'},
  {'name': 'ivysaur', 'url': 'https://pokeapi.co/api/v2/pokemon/2/'},
  {'name': 'venusaur', 'url': 'https://pokeapi.co/api/v2/pokemon/3/'},
  {'name': 'charmander', 'url': 'https://pokeapi.co/api/v2/pokemon/4/'},
  {'name': 'charmeleon', 'url': 'https://pokeapi.co/api/v2/pokemon/5/'},
  {'name': 'charizard', 'url': 'https://pokeapi.co/api/v2/pokemon/6/'},
  {'name': 'squirtle', 'url': 'https://pokeapi.co/api/v2/pokemon/7/'},
  {'name': 'wartortle', 'url': 'https://pokeapi.co/api/v2/pokemon/8/'},
  {'name': 'blastoise', 'url': 'https://pokeapi.co/api/v2/pokemon/9/'},
  {'name': 'caterpie', 'url': 'https://pokeapi.co/api/v2/pokemon/10/'},
  {'name': 'metapod', 'url': 'https://pokeapi.co/api/v2/pokemon/11/'},
  {'name': 'butterfree', 'url': 'https://pokeapi.co/api/v2/pokemon/12/'},
  {'name': 'weedle', '

We can see that we only have the name of the pokemons, we need more information, this dataset doesn't work for us

##### 4. Let's do some web scrapping...

In [14]:
from bs4 import BeautifulSoup

url = 'https://pokemondb.net/pokedex/all' #Fine page I found
page = requests.get(url)

if page.status_code == 200:
    soup = BeautifulSoup(page.content, 'lxml')
    
soup

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<title>Pokémon Pokédex: list of Pokémon with stats | Pokémon Database</title>
<link href="https://img.pokemondb.net" rel="preconnect"/>
<style>@font-face{font-family:"Fira Sans";font-style:normal;font-weight:400;font-display:swap;src:url("/static/fonts/fira-sans-v10-latin-400.woff2") format("woff2");unicode-range:U+0000-00FF,U+0131,U+0152-0153,U+02BB-02BC,U+02C6,U+02DA,U+02DC,U+2000-206F,U+2074,U+20AC,U+2122,U+2191,U+2193,U+2212,U+2215,U+FEFF,U+FFFD}@font-face{font-family:"Fira Sans";font-style:italic;font-weight:400;font-display:swap;src:url("/static/fonts/fira-sans-v10-latin-400i.woff2") format("woff2");unicode-range:U+0000-00FF,U+0131,U+0152-0153,U+02BB-02BC,U+02C6,U+02DA,U+02DC,U+2000-206F,U+2074,U+20AC,U+2122,U+2191,U+2193,U+2212,U+2215,U+FEFF,U+FFFD}@font-face{font-family:"Fira Sans";font-style:normal;font-weight:700;font-display:swap;src:url("/static/fonts/fira-sans-v10-latin-600.woff2") format("woff2");unicode-rang

##### 5. Nice page, now let's collect the data

In [20]:
#First collect all the names (We could do the pokedex index but will do it later)
names = soup.find_all('a', class_="ent-name")
names_s = list()

#Watch out with the special names like mega or alola form
special_names = soup.find_all('small', class_="text-muted")

cont=0 #This variable will iterate through special_names so when we find a repeated name we change it with his special name
for name in names:
    if name.text in names_s:
        names_s.append(name.text + '('+ special_names[cont].text + ')')
        cont += 1
    else:
        names_s.append(name.text)
        
names_s = pd.Series(names_s)
names_s

0                          Bulbasaur
1                            Ivysaur
2                           Venusaur
3            Venusaur(Mega Venusaur)
4                         Charmander
                    ...             
1070        Basculegion(Sensu Style)
1071                        Sneasler
1072                        Overqwil
1073                        Enamorus
1074    Enamorus(Own Tempo Rockruff)
Length: 1075, dtype: object

In [22]:
#Now collect the pokemon types
types = soup.find_all('td', class_="cell-icon")
type_1_s= list()
type_2_s= list()

for type in types:
    type = type.text.split()#The pokemons can have 1 type or 2
    type_1_s.append(type[0])
    if len(type) == 2:
        type_2_s.append(type[1])
    else:
        type_2_s.append(None)
        
type_1_s = pd.Series(type_1_s)
type_2_s = pd.Series(type_2_s)

pokemon_df = pd.DataFrame({'Name': names_s, 'Type 1': type_1_s, 'Type 2': type_2_s})
pokemon_df.to_csv('patata.csv')

,Name,Type 1,Type 2
0,Bulbasaur,Grass,Poison
1,Ivysaur,Grass,Poison
2,Venusaur,Grass,Poison
3,Venusaur(Mega Venusaur),Grass,Poison
4,Charmander,Fire,None
...,...,...,...
1070,Basculegion(Sensu Style),Water,Ghost
1071,Sneasler,Poison,Fighting
1072,Overqwil,Dark,Poison
1073,Enamorus,Fairy,Flying


In [ ]:
#We have a 'Total' column in the page but I'll try to do it myself (Is dumb so im going to grab it now)
#Stats
stats = soup.find_all('td', class_=re.compile(r"cell-num$")) #Idk why the reg doesn't work
stats_list= list()
#We create a list from all the numbers we got
for stat in stats:
    stats_list.append(stat.text)
    
#There are 8 columns with cell-nums so we need to take 8 steps
total = stats_list[1::8]
hp = stats_list[2::8] #The third colums is the hp column
attack = stats_list[3::8] #The fourth colums is the attack column
defense = stats_list[4::8] #The fifth colums is the defense column
sp_atk = stats_list[5::8] #The sixth colums is the sp_atk column
sp_def = stats_list[6::8] #The seventh colums is the sp_def column
speed = stats_list[7::8] #The eighth colums is the speed column

In [ ]:
#####  AQUI ES DONDE NO COJE solo cell-num y me agarra todas las que lo contienen


stats = soup.find_all('td', class_=re.compile(r"cell-num$"))
stats
#stats[1].text

[<td class="cell-num cell-fixed" data-sort-value="1"><span class="infocard-cell-img"><img alt="Bulbasaur" class="img-fixed icon-pkmn" height="42" loading="lazy" src="https://img.pokemondb.net/sprites/sword-shield/icon/bulbasaur.png" width="56"/></span><span class="infocard-cell-data">001</span></td>,
 <td class="cell-num cell-total">318</td>,
 <td class="cell-num">45</td>,
 <td class="cell-num">49</td>,
 <td class="cell-num">49</td>,
 <td class="cell-num">65</td>,
 <td class="cell-num">65</td>,
 <td class="cell-num">45</td>,
 <td class="cell-num cell-fixed" data-sort-value="2"><span class="infocard-cell-img"><img alt="Ivysaur" class="img-fixed icon-pkmn" height="42" loading="lazy" src="https://img.pokemondb.net/sprites/sword-shield/icon/ivysaur.png" width="56"/></span><span class="infocard-cell-data">002</span></td>,
 <td class="cell-num cell-total">405</td>,
 <td class="cell-num">60</td>,
 <td class="cell-num">62</td>,
 <td class="cell-num">63</td>,
 <td class="cell-num">80</td>,
 <td